In [11]:
import numpy as np
import theano.tensor as T
from theano import function
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = function([x,y],z)
print np.allclose(f(2,3),7)
print x.type

False
TensorType(float64, scalar)


In [12]:
x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y
f = function([x,y],z)

In [18]:
f(np.array([[1, 2], [3, 4]]), np.array([[10, 20], [30, 40]]))

array([[ 11.,  22.],
       [ 33.,  44.]])

In [30]:
a = T.vector('a')
b = T.vector('b')
z = a**2 + b**2 + 2*a*b
f = function([a,b],z)
f(np.array([1, 2,4]), np.array([4, 5,5]))

array([ 25.,  49.,  81.])

In [31]:
#sigmoid function
x = T.dmatrix('x')
sig = 1/(1+T.exp(-x))
log = function([x],sig)

In [32]:
log([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

In [34]:
tan = (1+T.tanh(x/2))/2
f = function([x],[sig,tan])

In [35]:
f([[0, 1], [-1, -2]])

[array([[ 0.5       ,  0.73105858],
        [ 0.26894142,  0.11920292]]), array([[ 0.5       ,  0.73105858],
        [ 0.26894142,  0.11920292]])]

In [36]:
s,t = f([[0, 1], [-1, -2]])

In [37]:
np.allclose(s,t)

True

In [38]:
from theano import shared

In [55]:
state = shared(0)
inc = T.iscalar('inc')
incr = function([inc],updates=[(state,state+inc)])
decr = function([inc],updates=[(state, state-inc)])

In [57]:
state.set_value(0)
incr(1)
print state.get_value()
incr(20)
print state.get_value()
decr(10)
print state.get_value()

1
21
11


In [74]:
state.set_value(0)
fn = state * 2 + inc
foo = T.scalar(dtype=state.dtype)
f = function([inc,foo],fn,givens=[(state,foo)])
print f(3,4)
print state.get_value()

float64
0


In [77]:
from theano.tensor.shared_randomstreams import RandomStreams

In [90]:
srng = RandomStreams(seed=1234)
r_u = srng.uniform((2,2))
r_n = srng.normal((2,2))
f_u = function([],r_u)
f_n = function([],r_n, no_default_updates=True)
print f_u()
print f_n()
close_zero = function([],[r_u+r_u - 2 * r_u,r_u])
print close_zero()

[[ 0.75718064  0.1130526 ]
 [ 0.00607781  0.8721389 ]]
[[ 0.42742609  1.74049825]
 [-0.02169041  1.48401086]]
[array([[ 0.,  0.],
       [ 0.,  0.]]), array([[ 0.31027075,  0.24150866],
       [ 0.56740797,  0.73226671]])]


### Logistic Regression Example

In [ ]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400                                # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

x = T.matrix('x')
y = T.vector('y')

w = theano.shared(rng.randn(feats),name='w')
b = theano.shared(0. , name = 'b')

print ('Initial model...\n')
print 'W=%s' %(w.get_value())
print 'b=%s' %(b.get_value())

sig = 1/(1 + T.exp(-T.dot(x,w)-b))
pred = sig > 0.5
crs_ent = -y * T.log(sig) - (1-y) * T.log(1-sig) 
cost = crs_ent.mean() + 0.01 * (w**2).sum()
gw , gb = T.grad(cost,[w,b])

train = theano.function(
                        inputs = [x,y],
                        outputs = [pred,crs_ent],
                        updates = [(w,w-0.01*gw),(b,b-0.01*gb)]
                       )
predict = theano.function(
                        inputs = [x],
                        outputs = [pred])

for i in range(training_steps):
    print 'Iteration %s' %(i)
    prediction,error = train(D[0],D[1])
    print 'Prediction = %s , Error = %s' %(prediction,error)

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))